In [83]:
# This notebook includes model developement with target variable as Stress_Level

In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [39]:
file_path = 'Data/Cleaned_Impact_of_Remote_Work_on_Mental_Health.csv'
df = pd.read_csv(file_path)

In [40]:
df['Mental_Health_Condition'].value_counts()

Mental_Health_Condition
Burnout       1280
Anxiety       1278
Depression    1246
Unknown       1196
Name: count, dtype: int64

In [41]:
df['Satisfaction_with_Remote_Work_Encoded'] = df['Satisfaction_with_Remote_Work'].apply(lambda x: 1 if x == 'Satisfied' else 0)

In [42]:
df['Sleep_Quality_Encoded'] = df['Sleep_Quality'].apply(lambda x: 2 if x == 'Good' else (1 if x == 'Average' else 0))

In [43]:
def classify_stress(row):
    if row['Stress_Level'] == 'High':
        return 1
    elif row['Stress_Level'] == 'Medium' and (row['Mental_Health_Condition'] != 'Unknown' or row['Sleep_Quality_Encoded'] == 0):
        return 1
    elif row['Stress_Level'] == 'Low' and (row['Mental_Health_Condition'] != 'Unknown' or row['Sleep_Quality_Encoded'] == 0):
        return 1
    else:
        return 0

df['Stress_Level_Binary'] = df.apply(classify_stress, axis = 1)

In [44]:
features_to_drop = ['Stress_Level', 'Employee_ID', 'Satisfaction_with_Remote_Work', 'Sleep_Quality', 'Region', 'Gender', 
                    'Job_Role', 'Stress_Level_Binary']

In [45]:
X = pd.get_dummies(df.drop(features_to_drop, axis = 1), drop_first = True)
y = df['Stress_Level_Binary']

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [47]:
class_distribution = y_train.value_counts()
print(class_distribution)

Stress_Level_Binary
1    3584
0     416
Name: count, dtype: int64


In [48]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state = 42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [49]:
class_distribution2 = y_train_resampled.value_counts()
print(class_distribution2)

Stress_Level_Binary
1    3584
0    3584
Name: count, dtype: int64


## Using XGBoost

In [57]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

def XGB_Bayesian(max_depth, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda):
    model = XGBClassifier(
        max_depth = int(max_depth),
        learning_rate = learning_rate,
        n_estimators = int(n_estimators),
        subsample = subsample,
        colsample_bytree = colsample_bytree,
        reg_alpha = reg_alpha,
        reg_lambda = reg_lambda,
        scale_pos_weight = 1,
        random_state = 42
    )
    
    # Performing cross-validation
    cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
    scores = cross_val_score(model, X_train, y_train, cv = cv, scoring = 'f1')
    return np.mean(scores)

param_bounds = {
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.1),
    'n_estimators': (50, 300),
    'subsample': (0.4, 1.0),
    'reg_alpha': (0.0, 1.0),
    'reg_lambda': (0.0, 1.0),
    'colsample_bytree': (0.6, 1.0)
}

optimizer = BayesianOptimization(
    f = XGB_Bayesian,
    pbounds = param_bounds,
    random_state = 42,
    verbose = 2
)

optimizer.maximize(init_points = 20, n_iter = 50)
print("Best Parameters:", optimizer.max)

|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
| 1         | 0.9607    | 0.7498    | 0.09556   | 8.124     | 199.7     | 0.156     | 0.156     | 0.4349    |
| 2         | 0.9668    | 0.9465    | 0.0641    | 7.957     | 55.15     | 0.9699    | 0.8324    | 0.5274    |
| 3         | 0.9663    | 0.6727    | 0.02651   | 5.13      | 181.2     | 0.4319    | 0.2912    | 0.7671    |
| 4         | 0.9656    | 0.6558    | 0.03629   | 5.565     | 164.0     | 0.7852    | 0.1997    | 0.7085    |
| 5         | 0.9545    | 0.837     | 0.01418   | 7.253     | 92.63     | 0.06505   | 0.9489    | 0.9794    |
| 6         | 0.9661    | 0.9234    | 0.03742   | 3.684     | 221.1     | 0.4402    | 0.122     | 0.6971    |
| 7         | 0.9625    | 0.6138    | 0.09184   | 4.811     | 215.6     | 0.3117    | 0.5201    | 0.728     |
| 8       

In [58]:
xgb_model = XGBClassifier(
    scale_pos_weight = 1,
    learning_rate = 0.016436662989208373,
    n_estimators = 222,
    max_depth = 4,
    colsample_bytree = 0.9800724956182749,
    subsample = 0.7919377274900352,
    reg_alpha = 0.48518241591646816,
    reg_lambda = 0.46933473866314557,
    random_state = 42
)

xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9800724956182749, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.016436662989208373,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=222, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [74]:
y_pred = xgb_model.predict(X_test)

print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.93      0.76        98
           1       0.99      0.94      0.97       902

    accuracy                           0.94      1000
   macro avg       0.81      0.94      0.86      1000
weighted avg       0.96      0.94      0.95      1000

Confusion Matrix:
 [[ 91   7]
 [ 52 850]]


In [91]:
from sklearn.metrics import classification_report, precision_recall_curve

y_probs = xgb_model.predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, y_probs)
f1_scores = 2 * (precision * recall) / (precision + recall)
best_threshold = thresholds[np.argmax(f1_scores)]

y_pred_adjusted = (y_probs >= best_threshold).astype(int)
print("\nClassification Report:\n", classification_report(y_test, y_pred_adjusted))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_adjusted))


Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.91      0.76        98
           1       0.99      0.95      0.97       902

    accuracy                           0.94      1000
   macro avg       0.82      0.93      0.87      1000
weighted avg       0.96      0.94      0.95      1000

Confusion Matrix:
 [[ 89   9]
 [ 46 856]]


## Using RandomForest

In [82]:
from sklearn.ensemble import RandomForestClassifier

def RandomForest_Bayesian(n_estimators, max_depth, min_samples_split, min_samples_leaf):
    model = RandomForestClassifier(
        max_depth = int(max_depth),
        n_estimators = int(n_estimators),
        min_samples_split = int(min_samples_split),
        min_samples_leaf = int(min_samples_leaf),
        random_state = 42
    )
    
    # Performing cross-validation
    cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
    scores = cross_val_score(model, X_train, y_train, cv = cv, scoring = 'f1')
    return np.mean(scores)

param_bounds = {
    'n_estimators': (50, 300),
    'max_depth': (3, 20),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 5)
}

optimizer = BayesianOptimization(
    f = RandomForest_Bayesian,
    pbounds = param_bounds,
    random_state = 42,
    verbose = 2
)

optimizer.maximize(init_points = 20, n_iter = 50)
print("Best Parameters:", optimizer.max)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.9654    | 9.367     | 4.803     | 7.856     | 199.7     |
| 2         | 0.9454    | 5.652     | 1.624     | 2.465     | 266.5     |
| 3         | 0.9648    | 13.22     | 3.832     | 2.165     | 292.5     |
| 4         | 0.962     | 17.15     | 1.849     | 3.455     | 95.85     |
| 5         | 0.9617    | 8.172     | 3.099     | 5.456     | 122.8     |
| 6         | 0.9642    | 13.4      | 1.558     | 4.337     | 141.6     |
| 7         | 0.9637    | 10.75     | 4.141     | 3.597     | 178.6     |
| 8         | 0.9646    | 13.07     | 1.186     | 6.86      | 92.63     |
| 9         | 0.9451    | 4.106     | 4.796     | 9.725     | 252.1     |
| 10        | 0.961     | 8.178     | 1.391     | 7.474     | 160.0     |
| 11        | 0.9455    | 5.075     | 2.981     | 2.275     | 277.3     |
| 12        | 0.9599    | 7.399     | 

In [87]:
rf_model = RandomForestClassifier(
    n_estimators = 63,
    max_depth = 19,
    min_samples_split = 2,
    min_samples_leaf = 4,
    random_state = 42,
)

rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=19, min_samples_leaf=4, n_estimators=63,
                       random_state=42)

In [88]:
y_pred2 = xgb_model.predict(X_test)

print("\nClassification Report:\n", classification_report(y_test, y_pred2))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred2))


Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.93      0.76        98
           1       0.99      0.94      0.97       902

    accuracy                           0.94      1000
   macro avg       0.81      0.94      0.86      1000
weighted avg       0.96      0.94      0.95      1000

Confusion Matrix:
 [[ 91   7]
 [ 52 850]]


In [89]:
y_probs2 = xgb_model.predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, y_probs)
f1_scores = 2 * (precision * recall) / (precision + recall)
best_threshold = thresholds[np.argmax(f1_scores)]

y_pred_adjusted2 = (y_probs >= best_threshold).astype(int)
print("\nClassification Report:\n", classification_report(y_test, y_pred_adjusted2))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_adjusted2))


Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.91      0.76        98
           1       0.99      0.95      0.97       902

    accuracy                           0.94      1000
   macro avg       0.82      0.93      0.87      1000
weighted avg       0.96      0.94      0.95      1000

Confusion Matrix:
 [[ 89   9]
 [ 46 856]]
